In [ ]:
!pip install rdflib

In [ ]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import RDF, RDFS, OWL, XSD
import json
import re

# Load existing ontology
g = Graph()

# Define namespaces
ex = Namespace("https://registry.gdi-de.org/id/de.bund.balm.radnet/")
geo = Namespace("http://www.opengis.net/ont/geosparql#")
g.bind("geo", geo)
g.bind("ex", ex)
g.parse("radnetz_ontology_v0.owl", format="application/rdf+xml")

# Define a function to create individuals
def add_individual(feature_uri, geom_uri, feature_type, coordinates, properties):
    # Ensure asWKT and hasGeometry properties exist
    ensure_property(geo.asWKT, OWL.DatatypeProperty, "asWKT")
    ensure_property(geo.hasGeometry, OWL.ObjectProperty, "hasGeometry")

    # Add the feature individual
    g.add((feature_uri, RDF.type, ex.SpatialObject))
    g.add((feature_uri, geo.hasGeometry, geom_uri))

    # Add the geometry individual
    g.add((geom_uri, RDF.type, geo.Geometry))
    wkt_literal = Literal(f"{feature_type} {coordinates}", datatype=geo.wktLiteral)
    g.add((geom_uri, geo.asWKT, wkt_literal))

    # Add properties
    for prop, value in properties.items():
        if prop == "stand":
            prop_uri = URIRef(ex["Datum"])
            date_literal = Literal(value, datatype=XSD.date)
            g.add((feature_uri, prop_uri, date_literal))
            ensure_property(prop_uri, OWL.DatatypeProperty, "Datum")
        elif prop == "FID":
            prop_uri = URIRef(ex["quell-ID"])
            g.add((feature_uri, prop_uri, Literal(value)))
            ensure_property(prop_uri, OWL.DatatypeProperty, "quell-ID")
        elif prop == "Routen-Nr":
            d_routes = [route.strip() for route in value.split(",") if re.match(r'D\d+', route.strip())]
            for d_route in d_routes:
                number = d_route[1:].lstrip("0")
                route_individual = URIRef(f"https://registry.gdi-de.org/codelist/de.bund.balm.radnetz/D-Route/{number}")
                g.add((feature_uri, ex['d-Route'], route_individual))
                ensure_property(ex['d-Route'], OWL.ObjectProperty, "d-Route")
        else:
            prop_uri = URIRef(ex[prop])
            g.add((feature_uri, prop_uri, Literal(value)))
            ensure_property(prop_uri, OWL.DatatypeProperty, prop)

def ensure_property(property_uri, property_type, label):
    if (property_uri, None, None) not in g:
        g.add((property_uri, RDF.type, property_type))
        g.add((property_uri, RDFS.label, Literal(label)))

# Load GeoJSON data
with open("c:\\Users\\49151\\Documents\\2023\\GitHub\\SpaLod\\src\\main\\python\\Radnetz_Deutschland_06-2022.geojson", 'r') as f:
    data = json.load(f)

# Iterate through the features in the JSON data
for i, feature in enumerate(data.get("features", []), start=1):  # Using .get() to avoid KeyError if "features" is not in data
    geometry = feature.get("geometry")
    if geometry is not None:  # Checking if geometry is not None
        feature_type = geometry.get("type")
        coordinates = geometry.get("coordinates")
        if feature_type is not None and coordinates is not None:
            # Create an URI for the feature
            feature_uri = URIRef(ex[f"feature{i}"])
            geom_uri = URIRef(ex[f"geom{i}"])

            # Extract properties
            properties = {k: v for k, v in feature.get("properties", {}).items() if v is not None}

            # Add them to the ontology
            add_individual(feature_uri, geom_uri, feature_type, coordinates, properties)

# Save the modified ontology
g.serialize(destination="modified_ontology.owl", format="application/rdf+xml")


-----

## Visualization

In [ ]:
!pip install pydeck rdflib


In [ ]:
!pip install pyproj


----

## Transform the data that is in the EPSG:25832 coordinates to EPSG:4326 corresponding to the latitude and longitude

In [ ]:
import pyproj

# Define the source and destination coordinate systems
# Here I'm assuming the source CRS is EPSG:25832, which is a common UTM zone for Germany.
# You will need to replace this with the correct EPSG code for your data.
source_crs = pyproj.CRS("EPSG:25832")  
dest_crs = pyproj.CRS("EPSG:4326")  # WGS84

# Create a transformer
transformer = pyproj.Transformer.from_crs(source_crs, dest_crs, always_xy=True)

def transform_coords(coords):
    return [transformer.transform(lon, lat) for lat, lon in coords]


---

## Read the ontologies and create an HTML map that draw lines and display properties in each line

In [ ]:
import pydeck as pdk
import rdflib
from rdflib import URIRef, Literal
from rdflib.namespace import RDF, OWL
import re

# Load the ontology
g = rdflib.Graph()
g.parse("modified_ontology.owl", format="application/rdf+xml")

# Define namespaces
ex = rdflib.Namespace("https://registry.gdi-de.org/id/de.bund.balm.radnet/")
geo = rdflib.Namespace("http://www.opengis.net/ont/geosparql#")

# Initialize Pydeck layer data
line_data = []
point_data = []

# Maximum number of features to display
MAX_FEATURES_TO_DISPLAY = 10000000

# Function to parse WKT format
import re

def parse_wkt(wkt_string):
    if wkt_string is None:
        print("Warning: WKT string is None")
        return []

    # Check if the WKT string is a LineString
    if wkt_string.lower().startswith("linestring"):
        coords_text = re.search(r'linestring \[\[(.+)\]\]', wkt_string, re.IGNORECASE)
        if coords_text is not None:
            coords = [tuple(map(float, coord.split(','))) for coord in coords_text.group(1).split('], [')]
            return [(lat, lon) for lon, lat, _ in coords]  # Convert (lon, lat, z) to (lat, lon) and ignore Z-coordinate
    else:
        print(f"Warning: Unexpected WKT type or could not parse WKT string: {wkt_string}")
    return []
# Iterate through features in the ontology
feature_count = 0
for feature in g.subjects(RDF.type, ex.SpatialObject):
    if feature_count >= MAX_FEATURES_TO_DISPLAY:
        break
    
    # Get geometry
    geom_uri = g.value(subject=feature, predicate=geo.hasGeometry)
    wkt_string = g.value(subject=geom_uri, predicate=geo.asWKT)
    coords = parse_wkt(str(wkt_string))
    coords = transform_coords(coords)
    # Get properties
    properties = {}
    for prop, value in g.predicate_objects(subject=feature):
        if isinstance(value, Literal):
            properties[str(prop.split('#')[-1])] = str(value)
    
    # For LineString, add data to line layer
    if "LINESTRING" in wkt_string.upper():
        print(properties)
        line_data.append({
            "start": coords[0],
            "end": coords[-1],
            "path": coords,
            "properties": str(properties)
        })
        feature_count += 1
    # For other geometries like Points, add data to point layer
    # You can add similar conditions for other geometries as needed
    else:
        print("not line: ",coords)
        for coord in coords:
            point_data.append({
                "position": coord,
                "properties": properties
            })
        feature_count += 1

# Define Pydeck layers
line_layer = pdk.Layer(
    'PathLayer',
    line_data,
    get_path="path",
    width_scale=20,
    width_min_pixels=2,
    get_color=[255, 0, 0, 255],
    pickable=True
)

point_layer = pdk.Layer(
    'ScatterplotLayer',
    point_data,
    get_position="position",
    get_radius=100,
    get_color=[0, 0, 255, 255],
    pickable=True
)

# Define view state
view_state = pdk.ViewState(latitude=51.1657, longitude=10.4515, zoom=6)

# Create Pydeck deck
r = pdk.Deck(layers=[line_layer, point_layer], initial_view_state=view_state, tooltip={"html": "{properties}", "style": {"color": "white"}})

# Render the deck to a standalone HTML file
r.to_html('map.html')


----
## Below is some other tests using folium but it was not enough efficient to display huge amount of data.

In [ ]:
import folium

# Create a map centered around a specific location
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Add a line manually
folium.PolyLine(locations=[(51.2, 10.4), (51.3, 10.5), (51.4, 10.6)], color="blue", weight=2.5, opacity=1).add_to(m)

# Save the map to an HTML file
m.save("map.html")


In [ ]:
!pip install folium shapely

In [ ]:
import json
import folium
from shapely.geometry import LineString
from folium.plugins import MarkerCluster

# Load GeoJSON data
with open("c:\\Users\\49151\\Documents\\2023\\GitHub\\SpaLod\\src\\main\\python\\Radnetz_Deutschland_06-2022.geojson", 'r') as f:
    data = json.load(f)

# Create a map centered around an average location
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Set a tolerance for line simplification
tolerance = 0.1

# Iterate through the features in the GeoJSON data
for feature in data.get("features", []):
    # Get the geometry and properties
    geometry = feature.get("geometry")
    properties = feature.get("properties")
    
    # Check if the geometry type is LineString
    if geometry and geometry.get("type") == "LineString":
        # Simplify the LineString
        coordinates = geometry.get("coordinates")
        line = LineString(coordinates)
        simplified_line = line.simplify(tolerance, preserve_topology=False)
        
        # Convert the coordinates for Folium and create a PolyLine
        folium_coords = [(lat, lon) for lon, lat, *_ in simplified_line.coords]
        folium.PolyLine(locations=folium_coords, color="blue", weight=2.5, opacity=1).add_to(m)

        # Prepare the popup text
        popup_text = "<br>".join([f"{k}: {v}" for k, v in properties.items() if v is not None])
        
        # Add a marker to the beginning of the line
        folium.Marker(location=folium_coords[0], popup=folium.Popup(popup_text, parse_html=True)).add_to(marker_cluster)

# Save the map to an HTML file
m.save("map.html")
